In [1]:
import numpy as np

import os
import pygame
import matplotlib
import time


# import sb3_contrib
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.ppo_mask import MaskablePPO

import matplotlib.backends.backend_agg as agg

# Import some environments which have different objectives and/or rewards function

import envRL_masking_obj1,envRL_masking_obj2, envRL_masking_obj3
import envRL_masking_obj4, envRL_masking_obj5, envRL_masking_obj8

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\tzhu1\Port-waterway-MLworkshop-RL-routing\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


{'all': NodeView(())}
FG1
Position(lat=30, long=15)
{'Cod': {'quantity': 100, 'timewindow': (1, 2)}}
FG2
Position(lat=30, long=50)
{'Cod': {'quantity': 500, 'timewindow': (1, 2)}}
FG3
Position(lat=60, long=50)
{'Cod': {'quantity': 300, 'timewindow': (1, 2)}}
S
Position(lat=10, long=10)
E
Position(lat=80, long=80)
FF1
Position(lat=50, long=60)
FF2
Position(lat=70, long=50)
{'all': [<main.FishGround object at 0x00000290CFFA6BA0>, <main.FishGround object at 0x00000290D0000CD0>, <main.FishGround object at 0x00000290D0001950>, <main.Start object at 0x00000290CFFA6CF0>, <main.End object at 0x00000290CFFA6E40>, <main.FishFactory object at 0x00000290CFFA6F90>, <main.FishFactory object at 0x00000290D0001A90>], 'fish ground': [<main.FishGround object at 0x00000290CFFA6BA0>, <main.FishGround object at 0x00000290D0000CD0>, <main.FishGround object at 0x00000290D0001950>], 'fish factory': [<main.FishFactory object at 0x00000290CFFA6F90>, <main.FishFactory object at 0x00000290D0001A90>], 'way point':

In [2]:
#Specify environment
envR=envRL_masking_obj1.envR
#Specify location for log and model file
Objective = 'Obj1'

In [3]:

episodes = 5
for episode in range(1, episodes +1):
    print('episode:{}'.format(episode))
    done = False
    obs, _ = envR.reset()
    envR.render()
    pygame.time.delay(1000)
    length = 10
    while not done:
        action = envR.action_space.sample(mask=envR.action_masks())
        print('action chosen:', action)
        obs, reward, done, truncated, info = envR.step(action)
        envR.render()
        pygame.time.delay(1000)
    print('episode:{} reward:{}'.format(episode, reward))
    matplotlib.pyplot.close()

envR.close()


episode:1
action chosen: 0
action chosen: 2
action chosen: 1
action chosen: 6
action chosen: 4
episode:1 reward:-1373.511082769788
episode:2
action chosen: 1
action chosen: 5
action chosen: 4
episode:2 reward:-3420.649768808303
episode:3
action chosen: 2
action chosen: 6
action chosen: 4
episode:3 reward:-1373.511082769788
episode:4
action chosen: 2
action chosen: 0
action chosen: 6
action chosen: 4
episode:4 reward:-1373.511082769788
episode:5
action chosen: 0
action chosen: 6
action chosen: 4
episode:5 reward:-1373.511082769788


In [4]:
# #Train an agent
# timesteps = [10000, 1000000, 3000000] # 20000, 100000, 1000000, 3000000, 5000000, 10000000
# for timestep in timesteps:
#     total_timesteps=timestep
#     start_time = time.time()
#     log_path = os.path.join('Training', 'Logs', f'{Objective}_{total_timesteps}stepModel')
#     model_path = os.path.join('Training', 'Model', f'{Objective}_{total_timesteps}stepModel')
#     model = MaskablePPO("MultiInputPolicy", envR, verbose=1, tensorboard_log=log_path, seed=100)
#     model.learn(total_timesteps=total_timesteps)
#     finish_time = time.time()
#     time_used = finish_time-start_time
#     print('Time spent for training:', time_used)
#     #save the trained model
#     model.save(model_path)

In [5]:
# Load a trained agent
total_timesteps=3000000
model_path = os.path.join('../Training', 'Model', f'{Objective}_{total_timesteps}stepModel')
model = MaskablePPO.load(model_path, env=envR)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
# Test the trained agent
vec_env = model.get_env()
obs  = vec_env.reset()
print('Initial observation',obs)
action, _ = model.predict(obs, action_masks=envR.action_masks())
print('chosen action by agent according to observation', action)
obs, reward, done, truncated = vec_env.step(action)
print('step output',obs, reward, done, truncated)

Initial observation OrderedDict({'pathDistance': array([[0.]]), 'route': array([3])})
chosen action by agent according to observation [0]
step output OrderedDict({'pathDistance': array([[2283.5887605]]), 'route': array([0])}) [-2283.5889] [False] [{'TimeLimit.truncated': False}]


In [9]:
# Run some episodes to see how the trained agent performs
episodes = 10

for episode in range(1, episodes + 1):
    print('episode:{}'.format(episode))
    done = False
    obs = vec_env.reset()
    envR.render()
    pygame.time.delay(1000)
    print('Observation output from initial',obs)
    length = 10
    while not done:
        print('input_obs', obs)
        action, _ = model.predict(obs, action_masks=envR.action_masks()) #, action_masks=envR.action_masks()
        print('action chosen:', action)
        obs, reward, done, truncated = vec_env.step(action)
        envR.render()
        pygame.time.delay(1000)
        print('output_obs', obs, reward, done, truncated) #observation is the same as state
    matplotlib.pyplot.close()
envR.close()

episode:1
Observation output from initial OrderedDict({'pathDistance': array([[0.]]), 'route': array([3])})
input_obs OrderedDict({'pathDistance': array([[0.]]), 'route': array([3])})
action chosen: [0]
output_obs OrderedDict({'pathDistance': array([[2283.5887605]]), 'route': array([0])}) [-2283.5889] [False] [{'TimeLimit.truncated': False}]
input_obs OrderedDict({'pathDistance': array([[2283.5887605]]), 'route': array([0])})
action chosen: [6]
output_obs OrderedDict({'pathDistance': array([[4955.20205843]]), 'route': array([6])}) [-4955.202] [False] [{'TimeLimit.truncated': False}]
input_obs OrderedDict({'pathDistance': array([[4955.20205843]]), 'route': array([6])})
action chosen: [4]
output_obs OrderedDict({'pathDistance': array([[0.]]), 'route': array([3])}) [-1373.5111] [ True] [{'episode': {'r': -8612.301902, 'l': 3, 't': 323.437078}, 'TimeLimit.truncated': False, 'terminal_observation': {'route': np.int64(4), 'pathDistance': 1373.511082769788}}]
episode:2
Observation output from